### Load the dataset and prepare the labels and sequences

In [ ]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

# Change working directory to Project folder
%cd "/content/drive/MyDrive/Machine_Learning_(CS-433)/Project_2"
# I assume this directory contains two folders: data and models, containing the data and models respectively.
file_path = 'data/dataset.txt'

# Load dataset
df = pd.read_csv(file_path, sep='\t')

#Pick just a random subset to make the processing to come run fast.
df = df.sample(n = 10)


Mounted at /content/drive
/content/drive/MyDrive/Machine_Learning_(CS-433)/Project_2


In [5]:
df.columns.values

array(['CHR', 'START', 'END', 'STRAND', 'GENE', 'TRANSCRIPT', 'IVS',
       'IVS_SIZE', 'BP_POS', 'BP_ACC_DIST', 'BP_ACC_SEQ', 'IVS_SEQ'],
      dtype=object)

In [6]:
df

,CHR,START,END,STRAND,GENE,TRANSCRIPT,IVS,IVS_SIZE,BP_POS,BP_ACC_DIST,BP_ACC_SEQ,IVS_SEQ
163835,chr20,37199226,37203884,+,RPN2,ENST00000237530,IVS4,4659,37203858,-27,ATTGGGGTTCTACTCTTTACCTTCCAG,GTGAGTTGTCATCTCGAGCATTTCTCAGGCTTCATTTGTCTCGGGT...
25246,chr2,131041463,131041814,+,ARHGEF4,ENST00000409359,IVS9,352,131041804,-11,TTCTGCCCCAG,GTAGGAGGGCACTGAGGCAGGGAGGCAGCCCACGCCTCTGCATGCC...
98831,chr11,60699776,60700859,+,MS4A8,ENST00000300226,IVS1,1084,60700827,-33,AGGGAAAATTCTCTCGCATTTATTTCTTGGCAG,GTGAGTCTGCTTAATCTGGTGGTTGGTAACTGGGAGTTGGTGATAA...
150634,chr18,57354525,57356912,+,ST8SIA3,ENST00000324000,IVS2,2388,57356892,-21,ATGAATTTCTTTTTATTTTAG,GTAGGATAGGAGGAAAAGATCCAAAAGGTGCTTTTAAGAATTCAAG...
66664,chr7,5923963,5925631,+,CCZ1,ENST00000325974,IVS14,1669,5925611,-21,ACTGTTGTGTCCTTGTTGCAG,GTAAGTAGGATTTGGTATTTCAGGAGAATTTGAGAAGTTGTCTTTT...
10600,chr1,150764765,150765697,-,CTSS,ENST00000368985,IVS1,933,150764795,-31,AACACTTCCTATGTTACCTTTCCTTACACAG,GTATGTACTCTTTGTTCTTTTTCTCCCATTTGGTCTTCAAACACTC...
4766,chr1,39292039,39292765,+,MACF1,ENST00000564288,IVS16,727,39292745,-21,ATTTCATTCTTTTTTAATCAG,GTGCGTAGCCTTCAGAATCCACATTACAGGAGGGACGTGGTTGGAA...
36295,chr3,56623567,56624478,-,TASOR,ENST00000683822,IVS23,912,56623599,-33,AATAGAGGAGGACTTTTTGTTTTTTAAATGTAG,GTAACTTTTCAGTGGTTTTCATTTTCTTTAACGCAGACAGAAAAAG...
41086,chr3,172760264,172761609,+,ECT2,ENST00000392692,IVS7,1346,172761588,-22,ACTGTAATGTTTATATTTTTAG,GTATGTAAACTTGGGTATTTTTGTGTATTTCAATACAGCATTATTT...
148124,chr17,81600428,81604547,-,NPLOC4,ENST00000331134,IVS8,4120,81600452,-25,ATCTTCTCCCTTTGATTCCCTGCAG,GTAAACATGACGGGATTCAAGTTCCTGAGTTTCTGTGTTTGGAGGC...


In [ ]:
df["IVS_SEQ"]

,IVS_SEQ
163835,GTGAGTTGTCATCTCGAGCATTTCTCAGGCTTCATTTGTCTCGGGT...
25246,GTAGGAGGGCACTGAGGCAGGGAGGCAGCCCACGCCTCTGCATGCC...
98831,GTGAGTCTGCTTAATCTGGTGGTTGGTAACTGGGAGTTGGTGATAA...
150634,GTAGGATAGGAGGAAAAGATCCAAAAGGTGCTTTTAAGAATTCAAG...
66664,GTAAGTAGGATTTGGTATTTCAGGAGAATTTGAGAAGTTGTCTTTT...
10600,GTATGTACTCTTTGTTCTTTTTCTCCCATTTGGTCTTCAAACACTC...
4766,GTGCGTAGCCTTCAGAATCCACATTACAGGAGGGACGTGGTTGGAA...
36295,GTAACTTTTCAGTGGTTTTCATTTTCTTTAACGCAGACAGAAAAAG...
41086,GTATGTAAACTTGGGTATTTTTGTGTATTTCAATACAGCATTATTT...
148124,GTAAACATGACGGGATTCAAGTTCCTGAGTTTCTGTGTTTGGAGGC...


In [36]:
#Create training data
input_sequences = df["IVS_SEQ"].to_list()
input_labels = []

for index, sample in df.iterrows():
    label_sequence = (sample["BP_POS"]-sample["START"])*"0" + "1" + (sample["END"]-sample["BP_POS"])*"0"
    input_labels.append(label_sequence)

In [9]:
input_sequences

['GTGAGTTGTCATCTCGAGCATTTCTCAGGCTTCATTTGTCTCGGGTCCTATCCGAAGAGGGCTCATTCATTGGTTCAGCAAATACTTTCTGGGCAAGTACTTGCAGTAAATACTTCCGTGTGCCAGGTGCTCTGCAAGGCACCGCAGACATGATTGCATGAAGGCAAGTAAGGTCCGTGTCCTCGTGGAGCTCACATGGACACAGTCTTCTGCAAGCCTCGCAGACAGAAGGCAAGCGCTGAGCCAGAGCATGAGCACACCACCACCTAGAGATCCAGTTGCCTTGTGCGGTTGGAGGGCTAGAGCAAGGGCAGATAACTGAGAAGGAGCCACGTGCATGTCTTGTTGAGGCTTGGCTAGGGCAGGATGACATGTTGGCAGTGGAGGCTCTGCAAAGTGGGGGCTCTCAGGATCCCTGGAAGTTGTGAAAGACAGCATCTGGTGGGTGAGGTGGTTGCTCAGAGGGTGGAGTATTCTAGGAATCAGACTCTACTACTCTGCAAAAGGCTTAGGTGAAGGAGAAGCACCTTTCCTATTTGTTATAAATGGAAATATTGTAACTTCAGGGTGTTTTTTGCTAAGCTCTTAAGTGATAATATATGTAGTTTTAAATTTTTATTTATTTAGAGACAGGATCTCACTCTGTCACCCAGGCTGAAGTGCAATGGTATGATCAAACTCACTGCAGCCTTGAACTCTGGGCTCAAGTGATTTTCCTACCTCAGCCTCCCGAGTAGCTGGACTACAGGTGCAAACTACCACGCCTAGCTAATTTTTTTTTTTTTTTGAGACGTGGTCTTACTCTGTTACCCAGGCTAGAGTGCAGTGACGCGATCTCAGTTCACTGCAACCTCCGCCTCCCGGGTTCAAGCAGTTCTCCTGCCTCAGCGTCTTCAGTGGCTGGGATTACAGGCGCGCCACCACGCCTGGCCAATTTTTGTATTTTTAGTAGAGACAGGGTTTCACCATGTTGGCCAGGTCTTGAACTCCTGACCTCA

In [10]:
input_labels

['00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

### Load the splicebert model and tokenizer

In [12]:
SPLICEBERT_PATH = "models/SpliceBERT.1024nt"  # set the path to the folder of pre-trained SpliceBERT
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForMaskedLM, AutoModelForTokenClassification

# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(SPLICEBERT_PATH)

# finetuning SpliceBERT for token classification tasks
model = AutoModelForTokenClassification.from_pretrained(SPLICEBERT_PATH, num_labels = 2) # We want binary classification on tokens so num_labels = 2

# use huggerface's official API to use SpliceBERT
# get nucleotide embeddings (hidden states)
#model = AutoModel.from_pretrained(SPLICEBERT_PATH) # load model
#last_hidden_state = model(input_ids).last_hidden_state # get hidden states from last layer
#hiddens_states = model(input_ids, output_hidden_states=True).hidden_states # hidden states from the embedding layer (nn.Embedding) and the 6 transformer encoder layers

# get nucleotide type logits in masked language modeling
#model = AutoModelForMaskedLM.from_pretrained(SPLICEBERT_PATH) # load model
#logits = model(input_ids).logits # shape: (batch_size, sequence_length, vocab_size)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at models/SpliceBERT.1024nt and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Show a case of inference

In [13]:
# prepare a test input to show how to do inference
seq = "ACGUACGuacguaCGuGCAUGUCAUGC"  ## WARNING: this is just a demo. SpliceBERT may not work on sequences shorter than 64nt as it was trained on sequences of 64-1024nt in length
seq = ' '.join(list(seq.upper().replace("U", "T"))) # U -> T and add whitespace
input_ids = tokenizer.encode(seq) # N -> 5, A -> 6, C -> 7, G -> 8, T(U) -> 9. NOTE: a [CLS] and a [SEP] token will be added to the start and the end of seq
input_ids = torch.as_tensor(input_ids) # convert python list to Tensor
input_ids = input_ids.unsqueeze(0) # add batch dimension, shape: (batch_size, sequence_length)

In [14]:
#infer with the (untrained) model
logit = model(input_ids, output_hidden_states=False).logits

In [27]:
#Define a function for making the output interpretable
def logittobinary(logits):
  """
  This is just a ghetto function for converting binary logits into binary strings
  """
  output_list = []
  for j in range(len(logits)):
    binary = ""
    for i in range(len(logits[j,:,1])):
      if logits[j,i,0] > logits[j,i,1]:
        binary += "1"
      else:
        binary += "0"
    output_list.append(binary)
  return output_list



In [28]:
logit = model(input_ids, output_hidden_states=False).logits

In [29]:
print(logittobinary(logit))

['00111011101110111010101011100']


The model is not yet finetuned so this output is just gibberish at the moment. Notice that for each token the likelyhood of a branchpoint is about the same as not a branchpoint, which should definately not be the case once the model has been trained.

### Tokenize the training data

In [37]:
#Tokenize the input data
input_sequences = [' '.join(list(seq.upper().replace("U", "T"))) for seq in input_sequences]
print(input_sequences)

['G T G A G T T G T C A T C T C G A G C A T T T C T C A G G C T T C A T T T G T C T C G G G T C C T A T C C G A A G A G G G C T C A T T C A T T G G T T C A G C A A A T A C T T T C T G G G C A A G T A C T T G C A G T A A A T A C T T C C G T G T G C C A G G T G C T C T G C A A G G C A C C G C A G A C A T G A T T G C A T G A A G G C A A G T A A G G T C C G T G T C C T C G T G G A G C T C A C A T G G A C A C A G T C T T C T G C A A G C C T C G C A G A C A G A A G G C A A G C G C T G A G C C A G A G C A T G A G C A C A C C A C C A C C T A G A G A T C C A G T T G C C T T G T G C G G T T G G A G G G C T A G A G C A A G G G C A G A T A A C T G A G A A G G A G C C A C G T G C A T G T C T T G T T G A G G C T T G G C T A G G G C A G G A T G A C A T G T T G G C A G T G G A G G C T C T G C A A A G T G G G G G C T C T C A G G A T C C C T G G A A G T T G T G A A A G A C A G C A T C T G G T G G G T G A G G T G G T T G C T C A G A G G G T G G A G T A T T C T A G G A A T C A G A C T C T A C T A C T C T 

In [32]:
input_sequences = [tokenizer.encode(seq) for seq in input_sequences]

In [35]:
input_sequences

[[2,
  8,
  9,
  8,
  6,
  8,
  9,
  9,
  8,
  9,
  7,
  6,
  9,
  7,
  9,
  7,
  8,
  6,
  8,
  7,
  6,
  9,
  9,
  9,
  7,
  9,
  7,
  6,
  8,
  8,
  7,
  9,
  9,
  7,
  6,
  9,
  9,
  9,
  8,
  9,
  7,
  9,
  7,
  8,
  8,
  8,
  9,
  7,
  7,
  9,
  6,
  9,
  7,
  7,
  8,
  6,
  6,
  8,
  6,
  8,
  8,
  8,
  7,
  9,
  7,
  6,
  9,
  9,
  7,
  6,
  9,
  9,
  8,
  8,
  9,
  9,
  7,
  6,
  8,
  7,
  6,
  6,
  6,
  9,
  6,
  7,
  9,
  9,
  9,
  7,
  9,
  8,
  8,
  8,
  7,
  6,
  6,
  8,
  9,
  6,
  7,
  9,
  9,
  8,
  7,
  6,
  8,
  9,
  6,
  6,
  6,
  9,
  6,
  7,
  9,
  9,
  7,
  7,
  8,
  9,
  8,
  9,
  8,
  7,
  7,
  6,
  8,
  8,
  9,
  8,
  7,
  9,
  7,
  9,
  8,
  7,
  6,
  6,
  8,
  8,
  7,
  6,
  7,
  7,
  8,
  7,
  6,
  8,
  6,
  7,
  6,
  9,
  8,
  6,
  9,
  9,
  8,
  7,
  6,
  9,
  8,
  6,
  6,
  8,
  8,
  7,
  6,
  6,
  8,
  9,
  6,
  6,
  8,
  8,
  9,
  7,
  7,
  8,
  9,
  8,
  9,
  7,
  7,
  9,
  7,
  8,
  9,
  8,
  8,
  6,
  8,
  7,
  9,
  7,
  6,
  7,
  6,
  9,
  8,
  8,


Active site
Active site


% df2 Data: 30.03% | % df2 Families: 32.32%:  32%|███▏      | 202/625 [00:00<00:01, 310.61it/s]


nrow df1: 1407
nrow df2: 604
Active site


% df2 Data: 53.15% | % df2 Families: 58.42%:  58%|█████▊    | 118/202 [00:00<00:00, 389.68it/s]


nrow df1: 283
nrow df2: 321


...

% df2 Data: 50.32% | % df2 Families: 60.33%:  60%|██████    | 111/184 [00:00<00:00, 376.42it/s]


nrow df1: 230
nrow df2: 233


Some weights of EsmForTokenClassification were not initialized from the model checkpoint at facebook/esm2_t12_35M_UR50D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Auc,Mcc
1,No log,0.113113,0.961334,0.823149,0.870667,0.846241,0.922312,0.824572
2,No log,0.102707,0.966662,0.861563,0.866422,0.863985,0.923520,0.844992
3,No log,0.099991,0.968536,0.863868,0.881440,0.872565,0.931051,0.854676
4,0.092400,0.104335,0.968683,0.891886,0.846334,0.868513,0.916025,0.851128
5,0.092400,0.101452,0.969248,0.871197,0.878208,0.874688,0.930066,0.857171


train metrics: {'accuracy': 0.988804113026122, 'precision': 0.9767630977922348, 'recall': 0.9853557215667109, 'f1': 0.9810405950229389, 'auc': 0.9877978090395455, 'mcc': 0.9731163303070904}
val metrics: {'accuracy': 0.9685355842948966, 'precision': 0.8638678425040368, 'recall': 0.8814397059755402, 'f1': 0.8725653169400621, 'auc': 0.9310506013843152, 'mcc': 0.8546758347037225}
test metrics: {'accuracy': 0.9820897875527177, 'precision': 0.9344536004739592, 'recall': 0.9426786199402336, 'f1': 0.9385480904468247, 'auc': 0.9657284293799832, 'mcc': 0.9280778313940207}
